Sliding Volume Filter (SVF)

$SVF_O = \frac{1}{M}\sum_{a=0}^{2\pi} \sum_{b=0}^{\pi} \max_{R_{min} < r < R_{max}}(\frac{1}{d+1}\sum_{r-d/2}^{r+d/2}VCI_Q)$

Voxel Convergence Index

$VCI_Q = \cos\phi(qx_\rho, qy_\rho, qz_\rho)
    = \cos\phi(\rho\sin{b}\cos{a}, \rho\sin{b}\sin{a}, \rho\cos{b})$

In [ ]:
def sliding_filter(rad, d, L, Rmax, Rmin, n, T):
    pass